## Be careful of the freeze state

In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from pretrained_resnet import Pre_trained_resnet

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
path = "/home/b/Desktop/Contrastive/ResNet50_pretrained_imagenet/Saved_models/regressor.pt"
trained_model = torch.load(path)
trained_model = trained_model.to(device)

In [5]:
# ground truths
ground_truths = np.load('/home/b/Desktop/Contrastive/Data/Steering_Angle/val_honda.npz')['val_targets'] 
print(ground_truths[0:100]) # Rotation angles

[-0.13944223  0.13280212 -0.2124834  -0.15936255 -0.00664011 -0.07304117
 -0.22576361 -0.07304117 -0.07304117 -0.24568393 -0.19920319 -0.07304117
  0.05312085 -0.23904382 -0.07304117 -0.01992032 -0.02656042  0.16600266
 -0.29216467 -0.30544489 -0.30544489 -0.30544489  0.13944223  0.65073041
 -0.91633466 -0.50464807 -0.41168659 -0.41168659 -0.41168659 -0.41168659
 -0.41168659 -0.41168659 -0.05976096  0.43160691 -0.19256308 -0.19256308
 -0.19256308 -0.19256308 -0.19256308 -0.19256308 -0.19256308 -0.19256308
 -0.19256308 -0.19256308 -0.37184595 -0.64409031 -0.13280212 -0.11288181
 -0.09296149 -0.04648074 -0.23904382 -0.27888446 -0.35192563 -0.46480744
 -0.19920319 -0.28552457 -1.00265604 -1.57370518 -1.68658699 -1.04913679
 -1.09561753 -0.91633466 -0.19256308 -0.17264276 -0.17928287 -0.15936255
 -0.07304117 -0.11952191 -0.22576361 -0.01992032 -0.17264276 -0.15936255
 -0.09296149 -0.02656042  0.          0.1062417   0.19256308 -0.13944223
 -0.11288181 -0.03984064 -0.04648074 -0.04648074 -0

In [6]:
def name_adv(x):
    return(float(x.split('/')[-1].split('_')[-1]))

def name_combined(x):
    return(int(x.split('/')[-1].split('_')[-2]))

def name_unseen(x):
    return(int(x.split('/')[-1].split('_')[-1]))

In [17]:
data_folders_combined = sorted(glob.glob('/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_combined/*'), key = name_combined)
data_folders_unseen = sorted(glob.glob('/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/*'), key = name_unseen)

In [18]:
data_folders_combined

['/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_combined/valHc_combined_1_0',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_combined/valHc_combined_2_0',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_combined/valHc_combined_3_0',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_combined/valHc_combined_4_0',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_combined/valHc_combined_5_0',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_combined/valHc_combined_6_0']

In [19]:
data_folders_unseen

['/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_frost_1',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_zoom_blur_1',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_motion_blur_1',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_snow_1',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_jpeg_compression_1',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_pixelate_1',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_fog_1',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_frost_2',
 '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_zoom_blur_2',
 '/home/b/Desktop/Contrastive/Data/Steering_Angl

In [30]:
data_folders_unseen = [
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_motion_blur_1',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_motion_blur_2',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_motion_blur_3',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_motion_blur_4',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_motion_blur_5',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_zoom_blur_1',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_zoom_blur_2',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_zoom_blur_3',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_zoom_blur_4',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_zoom_blur_5',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_pixelate_1',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_pixelate_2',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_pixelate_3',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_pixelate_4',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_pixelate_5',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_jpeg_compression_1',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_jpeg_compression_2',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_jpeg_compression_3',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_jpeg_compression_4',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_jpeg_compression_5',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_snow_1',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_snow_2',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_snow_3',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_snow_4',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_snow_5',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_frost_1',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_frost_2',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_frost_3',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_frost_4',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_frost_5',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_fog_1',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_fog_2',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_fog_3',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_fog_4',
    '/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_unseen/valB_IMGC_fog_5']

In [21]:
def name_adv_load(x): 
    return(int(x.split('/')[-1].split('.')[-2]))

class DriveDatasetNoLabels(Dataset):
    def __init__(self, images):
        self.images_list = images
        
    def __len__(self):
        return len(self.images_list)
    
    def __getitem__(self, key):
        image_idx = self.images_list[key]
        return image_idx.astype(np.float32)
    
def loss_mse(gt, pred):
    # Both in rotation angles, valid_loss = 0.938
    mse = torch.mean(torch.Tensor((gt - pred)**2))
    return mse

def accuracy_metrics(ground_truths, predictions):
    error = (ground_truths - predictions)*15.0

    acc_1 = 100*(np.sum(np.asarray([ 1.0 if er <=1.5 else 0.0 for er in error]))/error.shape[0])
    acc_2 = 100*(np.sum(np.asarray([ 1.0 if er <=3.0 else 0.0 for er in error]))/error.shape[0])
    acc_3 = 100*(np.sum(np.asarray([ 1.0 if er <=7.5 else 0.0 for er in error]))/error.shape[0])
    acc_4 = 100*(np.sum(np.asarray([ 1.0 if er <=15.0 else 0.0 for er in error]))/error.shape[0])
    acc_5 = 100*(np.sum(np.asarray([ 1.0 if er <=30.0 else 0.0 for er in error]))/error.shape[0])
    acc_6 = 100*(np.sum(np.asarray([ 1.0 if er <=75.0 else 0.0 for er in error]))/error.shape[0])

    mean_acc = (acc_1 + acc_2 + acc_3 + acc_4 + acc_5 + acc_6)/6
    return acc_1, acc_2, acc_3, acc_4, acc_5, acc_6, mean_acc   

In [31]:
#Make predictions from images
def make_predictions(all_image_paths,trained_model):
    images =[]
    #print()
    for i in range(len(all_image_paths)):
        try: 
            path = all_image_paths[i]
            images.append(np.asarray(Image.open(path)))

        except PIL.UnidentifiedImageError:
            print("Problem!!:",i)
            
    images = np.asarray(images)
    #print(images.shape)
    images = np.moveaxis(images,-1,1)
    print("Loaded {} images, Max = {}, Min = {}".format(images.shape, np.max(images), np.min(images)))
    
    BATCH_SIZE = 1
    adv_set = DriveDatasetNoLabels(images)

    adv_loader = torch.utils.data.DataLoader(dataset=adv_set,
                                                   batch_size=BATCH_SIZE,
                                                   collate_fn=None,
                                                   shuffle=False)

    predictions=[]
    with torch.no_grad():
        for bi, img in enumerate(adv_loader):
            inpt = img.to(device)
            outputs = trained_model(inpt) # input should be [1, 3, 66, 200]
            predictions.append(outputs.data.item())
            
    print("Predicted {} labels".format(len(predictions)))
    return predictions

In [23]:
txt_data=[]
#predictions on clean
clean_paths = sorted(glob.glob('/home/b/Desktop/Contrastive/Data/Steering_Angle/test_Data/validation_sets_clean/valHc/*.jpg'), key = name_adv_load)
#print(clean_path)
predictions_clean = make_predictions(clean_paths, trained_model)
acc_1, acc_2, acc_3, acc_4, acc_5, acc_6, mean_acc = accuracy_metrics(ground_truths, predictions_clean)
print("Acc:\n 1.5= {}%\n 3.0= {}%\n 7.5= {}%\n 15 = {}%\n 30= {}%\n 75 = {}%\nMean Accuracy = {}%".format(acc_1, 
                                                                                                   acc_2,
                                                                                                   acc_3,
                                                                                                   acc_4,
                                                                                                   acc_5,
                                                                                                   acc_6,
                                                                                                   mean_acc))
loss = loss_mse(ground_truths, predictions_clean)
print("Loss:", loss)
txt_data.append("Clean:    " + "Acc: " + str(mean_acc) + "    Loss:" + str(loss))


Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0


/home/b/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Predicted 10000 labels
Acc:
 1.5= 82.89999999999999%
 3.0= 91.64%
 7.5= 96.92%
 15 = 98.47%
 30= 99.18%
 75 = 99.77000000000001%
Mean Accuracy = 94.81333333333333%
Loss: tensor(0.4060)


In [ ]:
# loss_collect = []
# acc_collect =[]

# predictions_fgsm_total=[]
# for i in range(len(data_folders_fgsm)): 
#     all_image_paths = sorted(glob.glob(data_folders_fgsm[i]+"/"+"*.jpg"), key = name_adv_load)
#     predictions = make_predictions(all_image_paths, trained_model)
#     predictions_fgsm_total.append(predictions)
    
#     acc_1, acc_2, acc_3, acc_4, acc_5, acc_6, mean_acc = accuracy_metrics(ground_truths, predictions)
#     print("For {}\nAcc:\n 1.5= {}%\n 3.0= {}%\n 7.5= {}%\n 15 = {}%\n 30= {}%\n 75 = {}%\nMean Accuracy = {}%".format(data_folders_fgsm[i].split('/')[-1],
#                                                                                                     acc_1, 
#                                                                                                    acc_2,
#                                                                                                    acc_3,
#                                                                                                    acc_4,
#                                                                                                    acc_5,
#                                                                                                    acc_6,
#                                                                                                        mean_acc))
#     loss = loss_mse(ground_truths, predictions)
#     print("Loss:", loss)
#     loss_collect.append(str(round(loss.item(),2)))
#     acc_collect.append(str(round(mean_acc,2)))
#     txt_data.append(str(data_folders_fgsm[i].split('/')[-1]) + "    " + "Acc: " + str(mean_acc) + "    Loss:" + str(loss))

In [ ]:
# print("& {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{}".format(acc_collect[0],
#                                                                                 loss_collect[0],
#                                                                                 acc_collect[1],
#                                                                                 loss_collect[1],
#                                                                                 acc_collect[2],
#                                                                                 loss_collect[2],
#                                                                                 acc_collect[3],
#                                                                                 loss_collect[3],
#                                                                                 acc_collect[4],
#                                                                                 loss_collect[4]))

In [ ]:
# predictions_fgsm_total = np.array(predictions_fgsm_total)
# print(predictions_fgsm_total.shape)

In [ ]:
# permutation = np.arange(1100,1600,1)
# fig, ax = plt.subplots(figsize=(16,5), dpi =150)
# ax.plot(ground_truths[permutation]*15.0) 
# ax.plot(predictions_fgsm_total[4][permutation]*15.0)
# ax.set_ylabel("Steering Angle (Degrees)", fontsize=12)
# ax.set_xlabel("Time Steps", fontsize=12)
# ax.legend(["Ground Truths","Predictions on FGSM" ], fontsize=12)
# ax.set_ylim([-100,100])

In [ ]:
# loss_collect = []
# acc_collect =[]

# predictions_pgd_total=[]
# for i in range(len(data_folders_pgd)): 
#     all_image_paths = sorted(glob.glob(data_folders_pgd[i]+"/"+"*.jpg"), key = name_adv_load)
#     predictions = make_predictions(all_image_paths, trained_model)
#     predictions_pgd_total.append(predictions)
    
#     acc_1, acc_2, acc_3, acc_4, acc_5, acc_6, mean_acc = accuracy_metrics(ground_truths, predictions)
#     print("For {}\nAcc:\n 1.5= {}%\n 3.0= {}%\n 7.5= {}%\n 15 = {}%\n 30= {}%\n 75 = {}%\nMean Accuracy = {}%".format(data_folders_pgd[i].split('/')[-1],
#                                                                                                     acc_1, 
#                                                                                                    acc_2,
#                                                                                                    acc_3,
#                                                                                                    acc_4,
#                                                                                                    acc_5,
#                                                                                                    acc_6,
#                                                                                                    mean_acc))
#     loss = loss_mse(ground_truths, predictions)
#     print("Loss:", loss)
#     loss_collect.append(str(round(loss.item(),2)))
#     acc_collect.append(str(round(mean_acc,2)))
#     txt_data.append(str(data_folders_pgd[i].split('/')[-1]) + "    " + "Acc: " + str(mean_acc) + "    Loss:" + str(loss))

In [ ]:
# print("& {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{}".format(acc_collect[0],
#                                                                                 loss_collect[0],
#                                                                                 acc_collect[1],
#                                                                                 loss_collect[1],
#                                                                                 acc_collect[2],
#                                                                                 loss_collect[2],
#                                                                                 acc_collect[3],
#                                                                                 loss_collect[3],
#                                                                                 acc_collect[4],
#                                                                                 loss_collect[4]))

In [ ]:
# predictions_pgd_total = np.array(predictions_pgd_total)
# print(predictions_pgd_total.shape)

In [ ]:
# permutation = np.arange(900,1600,1)
# fig, ax = plt.subplots(figsize=(16,5), dpi =150)
# ax.plot(ground_truths[permutation]*15.0) 
# ax.plot(predictions_pgd_total[4][permutation]*15.0)
# ax.set_ylabel("Steering Angle (Degrees)", fontsize=14)
# ax.set_xlabel("Time Steps",fontsize=14)
# ax.legend(["Ground Truths","Predictions on FGSM" ],fontsize=14)
# ax.set_ylim([-100,100])

In [24]:
loss_collect = []
acc_collect =[]
predictions_combined_total=[]
for i in range(len(data_folders_combined)): 
    all_image_paths = sorted(glob.glob(data_folders_combined[i]+"/"+"*.jpg"), key = name_adv_load)
    predictions = make_predictions(all_image_paths, trained_model)
    predictions_combined_total.append(predictions)
    
    acc_1, acc_2, acc_3, acc_4, acc_5, acc_6, mean_acc = accuracy_metrics(ground_truths, predictions)
    print("For {}\nAcc:\n 1.5= {}%\n 3.0= {}%\n 7.5= {}%\n 15 = {}%\n 30= {}%\n 75 = {}%\nMean Accuracy = {}%".format(data_folders_combined[i].split('/')[-1],
                                                                                                    acc_1, 
                                                                                                   acc_2,
                                                                                                   acc_3,
                                                                                                   acc_4,
                                                                                                   acc_5,
                                                                                                   acc_6,
                                                                                                   mean_acc))
    loss = loss_mse(ground_truths, predictions)
    print("Loss:", loss)
    loss_collect.append(str(round(loss.item(),2)))
    acc_collect.append(str(round(mean_acc,2)))
    txt_data.append(str(data_folders_combined[i].split('/')[-1]) + "    " + "Acc: " + str(mean_acc) + "    Loss:" + str(loss))


Loaded (10000, 3, 66, 200) images, Max = 172, Min = 0
Predicted 10000 labels
For valHc_combined_1_0
Acc:
 1.5= 83.89999999999999%
 3.0= 86.78%
 7.5= 91.58%
 15 = 94.43%
 30= 96.47%
 75 = 98.14%
Mean Accuracy = 91.88333333333333%
Loss: tensor(8.4749)

Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0
Predicted 10000 labels
For valHc_combined_2_0
Acc:
 1.5= 50.5%
 3.0= 59.75%
 7.5= 80.66%
 15 = 90.68%
 30= 95.19999999999999%
 75 = 97.67%
Mean Accuracy = 79.07666666666667%
Loss: tensor(8.3673)

Loaded (10000, 3, 66, 200) images, Max = 226, Min = 0
Predicted 10000 labels
For valHc_combined_3_0
Acc:
 1.5= 45.59%
 3.0= 56.75%
 7.5= 80.42%
 15 = 90.60000000000001%
 30= 95.48%
 75 = 97.7%
Mean Accuracy = 77.75666666666667%
Loss: tensor(8.2444)

Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0
Predicted 10000 labels
For valHc_combined_4_0
Acc:
 1.5= 30.29%
 3.0= 38.42%
 7.5= 65.64%
 15 = 88.86%
 30= 95.21%
 75 = 97.67%
Mean Accuracy = 69.34833333333334%
Loss: tensor(8.4318)

Loaded (1

In [26]:
print("& {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{}".format(acc_collect[0],
                                                                                loss_collect[0],
                                                                                acc_collect[1],
                                                                                loss_collect[1],
                                                                                acc_collect[2],
                                                                                loss_collect[2],
                                                                                acc_collect[3],
                                                                                loss_collect[3],
                                                                                acc_collect[4],
                                                                                loss_collect[4],
                                                                                acc_collect[5],
                                                                                loss_collect[5]))

& 91.88\% /8.47 & 79.08\% /8.37 & 77.76\% /8.24 & 69.35\% /8.43 & 84.29\% /7.28 & 70.96\% /8.49


In [32]:
loss_collect = []
acc_collect =[]
predictions_unseen_total=[]
for i in range(len(data_folders_unseen)): 
    all_image_paths = sorted(glob.glob(data_folders_unseen[i]+"/"+"*.jpg"), key = name_adv_load)
    predictions = make_predictions(all_image_paths, trained_model)
    predictions_unseen_total.append(predictions)
    
    acc_1, acc_2, acc_3, acc_4, acc_5, acc_6, mean_acc = accuracy_metrics(ground_truths, predictions)
    print("For {}\nAcc:\n 1.5= {}%\n 3.0= {}%\n 7.5= {}%\n 15 = {}%\n 30= {}%\n 75 = {}%\nMean Accuracy = {}%".format(data_folders_unseen[i].split('/')[-1],
                                                                                                    acc_1, 
                                                                                                   acc_2,
                                                                                                   acc_3,
                                                                                                   acc_4,
                                                                                                   acc_5,
                                                                                                   acc_6,
                                                                                                   mean_acc))
    loss = loss_mse(ground_truths, predictions)
    print("Loss:", loss)
    loss_collect.append(str(round(loss.item(),2)))
    acc_collect.append(str(round(mean_acc,2)))
    txt_data.append(str(data_folders_unseen[i].split('/')[-1]) + "    " + "Acc: " + str(mean_acc) + "    Loss:" + str(loss))

Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0
Predicted 10000 labels
For valB_IMGC_motion_blur_1
Acc:
 1.5= 71.78%
 3.0= 84.86%
 7.5= 94.66%
 15 = 97.35000000000001%
 30= 98.66%
 75 = 99.52%
Mean Accuracy = 91.13833333333332%
Loss: tensor(1.0141)
Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0
Predicted 10000 labels
For valB_IMGC_motion_blur_2
Acc:
 1.5= 67.78999999999999%
 3.0= 81.39999999999999%
 7.5= 93.01%
 15 = 96.32%
 30= 98.16%
 75 = 99.39%
Mean Accuracy = 89.34499999999998%
Loss: tensor(1.5025)
Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0
Predicted 10000 labels
For valB_IMGC_motion_blur_3
Acc:
 1.5= 64.05%
 3.0= 76.98%
 7.5= 90.59%
 15 = 95.24000000000001%
 30= 97.23%
 75 = 98.92%
Mean Accuracy = 87.16833333333334%
Loss: tensor(3.0233)
Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0
Predicted 10000 labels
For valB_IMGC_motion_blur_4
Acc:
 1.5= 58.989999999999995%
 3.0= 71.05%
 7.5= 86.96000000000001%
 15 = 93.5%
 30= 96.39999999999999%
 75 = 98.6199

Predicted 10000 labels
For valB_IMGC_fog_3
Acc:
 1.5= 44.39%
 3.0= 52.96999999999999%
 7.5= 69.69999999999999%
 15 = 82.66%
 30= 91.64999999999999%
 75 = 97.15%
Mean Accuracy = 73.08666666666666%
Loss: tensor(8.8199)
Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0
Predicted 10000 labels
For valB_IMGC_fog_4
Acc:
 1.5= 41.47%
 3.0= 50.019999999999996%
 7.5= 69.05%
 15 = 82.35%
 30= 91.72%
 75 = 97.17%
Mean Accuracy = 71.96333333333334%
Loss: tensor(8.7192)
Loaded (10000, 3, 66, 200) images, Max = 255, Min = 0
Predicted 10000 labels
For valB_IMGC_fog_5
Acc:
 1.5= 46.67%
 3.0= 54.37%
 7.5= 71.98%
 15 = 84.21%
 30= 92.10000000000001%
 75 = 97.06%
Mean Accuracy = 74.39833333333333%
Loss: tensor(9.0965)


In [35]:
for j in range(1,len(data_folders_unseen)+1):
    if j%5==0:
        
        print(j-5, j-4, j-3, j-2, j-1)
        print(j)
#         print(data_folders_unseen[j])
        print("& {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{} & {}\% /{}\n".format(acc_collect[j-5],
                                                                                        loss_collect[j-5],
                                                                                        acc_collect[j-4],
                                                                                        loss_collect[j-4],
                                                                                        acc_collect[j-3],
                                                                                        loss_collect[j-3],
                                                                                        acc_collect[j-2],
                                                                                        loss_collect[j-2],
                                                                                        acc_collect[j-1],
                                                                                        loss_collect[j-1]))
        

0 1 2 3 4
5
& 91.14\% /1.01 & 89.34\% /1.5 & 87.17\% /3.02 & 84.25\% /4.17 & 82.04\% /5.21

5 6 7 8 9
10
& 87.42\% /3.62 & 85.69\% /5.11 & 83.49\% /6.26 & 82.7\% /6.9 & 81.78\% /7.19

10 11 12 13 14
15
& 92.7\% /0.62 & 92.68\% /0.65 & 92.08\% /0.69 & 91.88\% /0.83 & 91.68\% /0.91

15 16 17 18 19
20
& 92.02\% /0.72 & 91.37\% /0.89 & 91.02\% /1.0 & 89.76\% /1.5 & 88.4\% /2.16

20 21 22 23 24
25
& 88.48\% /4.67 & 86.99\% /6.97 & 84.52\% /7.71 & 83.49\% /8.15 & 75.35\% /8.24

25 26 27 28 29
30
& 83.73\% /3.18 & 78.31\% /6.76 & 75.47\% /7.97 & 76.69\% /8.83 & 75.46\% /9.46

30 31 32 33 34
35
& 76.69\% /7.49 & 75.23\% /7.79 & 73.09\% /8.82 & 71.96\% /8.72 & 74.4\% /9.1



In [ ]:
print(len(data_folders_unseen))
print(data_folders_unseen)

In [ ]:
# fname= "/home/b/Desktop/week_3_experiments/results/" + path.split('/')[-1] + "_results.txt"
# np.savetxt(fname, txt_data, fmt='%s', newline='\n')